## Exploratory data analysis with Pandas and Seaborn

`Pandas` is a Python library that provides extensive means for data analysis. Data scientists often work with data stored in table formats like .csv, .tsv, or .xlsx. Pandas makes it very convenient to load, process, and analyze such tabular data. Together with `Matplotlib` and `Seaborn`, `Pandas` provides a wide range of opportunities for visual analysis of tabular data.

The main data structures in `Pandas` are implemented with **Series** and **DataFrame** classes. The former is a one-dimensional indexed array of some fixed data type. The latter is a two-dimensional data structure — a table — where each column contains data of the same type. DataFrames are great for representing real data: rows correspond to instances (examples, observations, etc.), and columns correspond to features of these instances.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# imports to set up plotting
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# some nice visual settings
pd.set_option("display.precision", 2)

# graphics in the "retina" format are more sharp and legible
sns.set()
%config InlineBackend.figure_format = 'retina'

# setting the plot size and color scheme
plt.rcParams["figure.figsize"] = (7, 4)
plt.rcParams["image.cmap"] = "viridis"

We will show the main methods in action by analyzing a [dataset on the churn rate](https://www.kaggle.com/datasets/spscientist/telecom-data) of telecom operator clients. A customer that is "churning" is a customer who terminates his/her contract with the company (typically because he/she is not satisfied with the service). The churn rate measures the operator's loss in customers.

### First look at the data

To load the data in the .csv format, we use the `read_csv()` method from Pandas. We then use the `head()` method to have a look at the first 5 lines of a new DataFrame (to make sure that everything is ok). (Note that you need to create a directory called "data" in your working directory (typically, it is the same directory where this notebook is located) and put telecom_churn.csv there.)

In [ ]:
df = pd.read_csv("data/telecom_churn.csv")
df.head()

Let’s look at the data more closely. Using `shape`, we can see that the dataset contains 3333 rows and 20 columns.

In [ ]:
df.shape

Using `columns`, we have access to column (feature) names.

In [ ]:
df.columns

We also can use the `info()` method to output some general information about the DataFrame. 

In [ ]:
df.info()

From the output, for example, we can easily see that there are no missing values in the dataset (because each column contains 3333 values, the same number as the total number of observations).

The method `describe()` shows some basic statistical characteristics for each numerical feature: number of non-missing values, mean, standard deviation, range, median, 0.25, and 0.75 quartiles.

In [ ]:
df.describe()

For categorical and boolean features, we can use `value_counts()`. Let's take a look at the distribution of *Churn*.

In [ ]:
df["Churn"].value_counts()

To calculate ratios, we can pass `normalize=True` to `value_counts()`.

In [ ]:
df["Churn"].value_counts(normalize=True)

**[Q1] What state is the most frequent US state in the dataset? How many different states are there in the dataset? Use `value_counts()` to answer these questions.**

In [ ]:
df["State"].value_counts() # -> WV    106 is the highest

### Indexing and retrieving data


A DataFrame can be indexed in a few different ways. To get a single column, we can use `DataFrame["Name"]`. Let's use this to calculate the proportion of customers who have churned (i.e., `Churn == True`)? We can calculate it directly, without `value_counts()`, as follows:

In [ ]:
df["Churn"].mean()

The rate of 14.5% is actually quite bad for a company; such a churn rate can make the company go bankrupt.

**[Q2] How much time, on average, do customers spend on the phone during daytime? Calculate mean and median values with the same type of command as above.**

In [ ]:
print(df["Total day minutes"].median()) # mean = 179.7 min, med =179.4

Another convenient and widely used way of indexing in DataFrames is boolean indexing. The syntax is `df[predicate]`, where `predicate` is some logical expression for one or several columns checked for each element of these columns. The result of such indexing is the DataFrame consisting only of rows that satisfy the predicate condition. 
  
Let's use this to calculate how much time, on average, the customers who have and have not churned spend on the phone during daytime.

In [ ]:
df[df["Churn"] == True]["Total day minutes"].mean(),\
df[df["Churn"] == False]["Total day minutes"].mean()

To see what is the maximum length of international calls among the customers that have not churned and do not have an international plan, we can use:

In [ ]:
df[(df["Churn"] == False) & (df["International plan"] == "No")]["Total intl minutes"].max()

**[Q3] What is the proportion of churned customers among those that have and do not have an international plan?**

In [ ]:
internationalPlan = df[(df["Churn"] == True) & (df["International plan"] == "Yes")].shape[0] / df[(df["International plan"] == "Yes")].shape[0]
noInternationalPlan = df[(df["Churn"] == True) & (df["International plan"] == "No")].shape[0] / df[(df["International plan"] == "No")].shape[0]
print("churned [%] with internationalPlan : " + str(internationalPlan))
print("churned [%] without internationalPlan : " + str(noInternationalPlan))

#churned proportion with internationalPlan : 0.4241486068111455
# churned proportion without internationalPlan : 0.11495016611295682

**[Q4] What is the proportion of churned customers among those that (1) have an international plan and (2) made at least 3 customer service calls?**

In [ ]:
internationalPlanAndSupport = df[(df["Churn"] == True) & (df["International plan"] == "Yes") & (df["Customer service calls"] >= 3)].shape[0] / df[(df["International plan"] == "Yes")].shape[0]
print("churned proportion with 3 support call and international plan : " + str(internationalPlanAndSupport))
#churned proportion with 3 support call and international plan : 0.10526315789473684

### Grouping data

What we did above can be done more effectively using grouping. In general, grouping data in Pandas works as follows:

```python
df.groupby(by=grouping_columns)[columns_to_show].function()
```

1. First, the `groupby()` method divides the `grouping_columns` by their values. The resulting DataFrame will be indexed by `grouping_columns`.
2. Then, columns of interest are selected (`columns_to_show`). If `columns_to_show` is not specified, all non-groupby columns are included.
3. Finally, one or several functions are applied to all the obtained groups (for each selected column).


Here is an example where we group the data according to the values of *Churn* and display various statistics of two columns in each group.

In [1]:
columns_to_show = ["Total day minutes", "Total intl minutes"]

df.groupby(["Churn"])[columns_to_show].describe()

NameError: name 'df' is not defined

In [ ]:
columns_to_show = ["Total day minutes", "Total intl minutes"]

df.groupby(["Churn"])[columns_to_show].agg([np.mean, np.std, np.min, np.max])

**[Q5] Display similarly the same statistics for *Total eve minutes* and *Total night minutes*.**

In [ ]:
# your codecolumns_to_show = ["Total eve minutes", "Total night minutes"]
print(df.groupby(["Churn"])[columns_to_show].describe())
print(df.groupby(["Churn"])[columns_to_show].agg([np.mean, np.std, np.min, np.max])) here

### Summary tables

Suppose we want to see how the observations in our dataset are distributed in the context of two variables — *Churn* and *International plan*. To do so, we can build a contingency table using `crosstab()`.
In statistics, a contingency table (also known as a cross tabulation or crosstab) is a type of table in a matrix format that displays how often a pair of variables take a specific value in a dataset. It provides a basic picture of the interrelation between two variables and can help find interactions between them.

In [ ]:
pd.crosstab(df["Churn"], df["International plan"])

**[Q6] Build a "normalized" contingency table for *Churn* and *Voice mail plan* with ratios instead of counts. To do this, pass `normalize=True` to `crosstab()`. Read more about other possible ways to normalize crosstabs at [Pandas documentation](https://pandas.pydata.org/docs/reference/api/pandas.crosstab.html).**

In [ ]:
print(pd.crosstab(df["Churn"], df["Voice mail plan"], normalize=True))


#Voice mail plan    No   Yes
#Churn
#False            0.60  0.25
#True             0.12  0.02

**[Q7] Build a contingency table for *Churn* and *Customer service calls*. Add row and column subtotals to this table. To find out how to do this, use the [Pandas documentation](https://pandas.pydata.org/docs/reference/api/pandas.crosstab.html).**

In [ ]:
print(pd.crosstab(df["Churn"], df["Customer service calls"], normalize=True,margins=True))

# Customer service calls     0     1     2     3     4         5         6         7         8         9   All
#Churn
#False                   0.18  0.32  0.20  0.12  0.03  7.80e-03  2.40e-03  1.20e-03  3.00e-04  0.00e+00  0.86
#True                    0.03  0.04  0.03  0.01  0.02  1.20e-02  4.20e-03  1.50e-03  3.00e-04  6.00e-04  0.14
#All                     0.21  0.35  0.23  0.13  0.05  1.98e-02  6.60e-03  2.70e-03  6.00e-04  6.00e-04  1.00

**Interesting observation:** We can see that most of the customers are loyal and do not use additional services (International Plan/Voice mail). It can also be seen that the churn rate increases sharply starting from 4 customer service calls.

### Visual analysis (and first attempt at predicting telecom churn)


Here we discuss a number of possible ways to visualize the data and the results we obtained above. For these purposes, we will use `Seaborn`.

To see how *Сhurn* is related to *International plan* we can use `countplot()`.

In [ ]:
sns.countplot(x="International plan", hue="Churn", data=df);

Here we clearly see that, with an international plan, the churn rate is much higher. Perhaps large and poorly controlled expenses with international calls are very conflict-prone and lead to dissatisfaction among customers.

**[Q8] Make a similar plot for *Customer service calls*.**

In [ ]:
sns.countplot(x="Customer service calls", hue="Churn", data=df).get_figure().savefig("out.png")

As we discussed earlier, the churn rate increases sharply starting from 4 customer service calls. It is much easier to notice it on the chart! 

Now let's add a new binary feature to our DataFrame: `Many customer service calls = Customer service calls > 3`.

In [ ]:
df["Many customer service calls"] = df["Customer service calls"] > 3

**[Q9] Using the crosstab and countplot, show how *Many_service_calls* relates to Churn.** 

In [ ]:
df["Many customer service calls"] = df["Customer service calls"] > 3
print(pd.crosstab(df["Churn"], df["Many customer service calls"], margins=True))
sns.countplot(x="Many customer service calls", hue="Churn", data=df).get_figure().savefig("out2.png")

#Many customer service calls  False  True   All
#Churn
#False                         2721   129  2850
#True                           345   138   483
#All                           3066   267  3333

Let’s make another contingency table that relates *Churn* with both *International plan* and *Many_service_calls*.

In [ ]:
pd.crosstab((df["Many customer service calls"]) & (df["International plan"]=='Yes'), df["Churn"])

**On the prediction of churned customers:** We can see from above that by predicting that a customer will terminate his/her contract with the company (i.e., *Churn*=1) in the case when (1) *Customer service calls* is greater than 3 and (2) the *International Plan* is True (and predicting *Churn*=0 otherwise), we might rarely be wrong. Namely, for the customers in our dataset, we will make a mistake only $464 + 9 = 473$ times. Speaking in terms of Machine Learning lingo, we might expect an accuracy of $1-473/3333=0.858$ (accuracy score is an evaluation metric that measures the number of correct predictions made by a model in relation to the total number of predictions made; it is calculated by dividing the number of correct predictions by the total number of predictions). This number, $85.8\%$, that we got through this simple reasoning can serve as a good starting point (baseline) for further models. 

### Other plots that might be useful

In what follows, we show other possible graphs that can be plotted with `Seaborn` and `Pandas`. 

Let's start with histograms. A histogram is a classic visualization tool that represents the distribution of one or more variables by counting the number of observations that fall within discrete bins. It is often used to illustrate the major features of the distribution of the data in a convenient form. The shape of a histogram provides clues about the underlying distribution type. To build a histogram for one or several features, we can use the method `hist()` directly from Pandas.

In [ ]:
columns_to_show = ["Total day minutes", "Total intl calls"]
df[columns_to_show].hist(figsize=(9,3));

**[Q10] Use the [Pandas documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.hist.html) to find how to change the number of bins in the histograms from above.**

In [ ]:

columns_to_show = ["Total day minutes", "Total intl calls"]
(df[columns_to_show].hist(figsize=(9,3), bins=10))[0][0].get_figure().savefig('out30.png')

To build histograms, we can also use `histplot()` from Seaborn. This function can normalize the statistic computed within each bin to estimate frequency, density, or probability mass (we specify it in `stat`).

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(9,3))

sns.histplot(data=df["Total day calls"], ax=axes[0], bins=20, stat="density");
sns.histplot(data=df["Total intl calls"], ax=axes[1], bins=10, stat="density");

fig.tight_layout()

One can also draw box plots for various features using Seaborn. 

In [ ]:
fig , axes = plt.subplots(1,2, figsize=(9,3))

sns.boxplot(data=df["Total day calls"], ax=axes[0]);
sns.boxplot(data=df["Total intl calls"], ax=axes[1]);

fig.tight_layout()

To explore the relationship between two features, we can use the scatter plot. The scatter plot graphs pairs of numerical data, with one variable on each axis, to look for a relationship between them. If the variables are correlated (we will discuss the correlation later in the course), the points will fall along a line or curve. The better the correlation, the tighter the points will hug the line. 

In Seaborn, `scatterplot()` can be used to make a scatter plot.

In [ ]:
sns.scatterplot(data=df, x="Total day minutes", y="Total night minutes");

Additionally, the points of a scatter plot can be color or size coded so that the values of a third categorical variable are also presented in the same figure. We can use the parameter `hue` to indicate our categorical feature of interest:

In [ ]:
sns.scatterplot(data = df, x="Total day minutes", y="Total night minutes", hue="Churn");

**[Q11] We can see that the size of markers in the scatter plots is quite large and this complicates their interpretation. Use the [Seaborn documentation](https://seaborn.pydata.org/generated/seaborn.scatterplot.html) or Google to find out how to change the marker size in the scatterplots.**

In [ ]:
sns.scatterplot(data = df, x="Total day minutes", y="Total night minutes", hue="Churn", size=1).get_figure().savefig("out41.png")

To visualize the correlation between several features in a DataFrame, we can use the correlation matrix. Loosely speaking, this matrix shows the correlation values, which measure the degree of linear relationship between each pair of features (but we will discuss it in more detail later in the course). The correlation matrix is used to summarize data, as input into a more advanced analysis, and as a diagnostic for advanced analyses.

Below, we first drop some of the features (calls and minutes duplicate charges), then we use the method `corr()` from Pandas that calculates the correlation between each pair of features (at this step, non-numerical features are excluded automatically), and finally, we pass the resulting correlation matrix to `heatmap()` from Seaborn, which renders a color-coded matrix for the provided values:

In [ ]:
all_features = set(df.columns)
features_to_exclude = set(["Total day calls",
                        "Total day minutes",
                        "Total eve calls",
                        "Total eve minutes",
                        "Total night calls",
                        "Total night minutes",
                        "Total intl calls",
                        "Total intl minutes"])

features_to_show = list(all_features-features_to_exclude)
corr_matrix = df[features_to_show].corr()
sns.heatmap(corr_matrix);

We can see from this chart that *Churn* correlates the most with *Total day charge* and *Customer service calls* (and our *Many customer service calls*).